# Анализ сервиса вопросов и ответов по программированию


**Описание проекта**

Проект выполнен на основе базы данных StackOverflow — сервиса вопросов и ответов о программировании. База содержит данные о постах, созданных в 2008 году.

**Задача:** с помощью SQL посчитать и визуализировать ключевые метрики сервис-системы вопросов и ответов о программировании.



**Описание данных** 
* `stackoverflow.badges` — хранит информацию о значках, которые присуждаются за разные достижения;
* `stackoverflow.post_types` — хранит информацию о типе постов;
* `stackoverflow.posts` — содержит информацию о постах;
* `stackoverflow.users` — содержит информацию о пользователях;
* `stackoverflow.vote_types` — содержит информацию о типах голосов;
* `stackoverflow.votes` — содержит информацию о голосах за посты.

# Написание запросов: первая часть

1. Количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».


In [ ]:
SELECT COUNT(*) AS question_count
FROM stackoverflow.posts
WHERE ((score > 300) OR (favorites_count >= 100))
  AND (post_type_id = 1);

2. Среднее количество вопросов в день с 1 по 18 ноября 2008 включительно.

In [ ]:
SELECT ROUND(AVG(question_count_per_day)) AS average_questions_per_day
FROM (
    SELECT date_trunc('day', creation_date)::date AS post_date, COUNT(*) AS question_count_per_day
    FROM stackoverflow.posts
    WHERE post_type_id = 1
        AND date_trunc('day', creation_date)::date BETWEEN '2008-11-01' AND '2008-11-18'
    GROUP BY post_date
) AS daily_question_counts;

3. Количество уникальных пользователей, которые получили значки сразу в день регистрации.


In [ ]:
SELECT COUNT(DISTINCT u.id) AS users_with_badges_on_registration_day
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE date_trunc('day', u.creation_date) = date_trunc('day', b.creation_date);

4. Количество уникальных постов пользователя с именем Joel Coehoorn с хотя бы одним голосом.


In [ ]:
SELECT COUNT(DISTINCT p.id) AS unique_posts_with_votes
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn';

5. Таблица vote_types с новым полем rank, в которое вошли номера записей в обратном порядке, с сортировкой по полю id.

In [ ]:
SELECT *, ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

6. 10 пользователей, которые поставили больше всего голосов типа 'Close': идентификаторы и количество голосов с сортировкой по убыванию количества голосов, затем по убыванию значения идентификатора пользователя.


In [ ]:
SELECT v.user_id, COUNT(*) AS vote_count
FROM stackoverflow.votes v
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY v.user_id
ORDER BY vote_count DESC, v.user_id DESC
LIMIT 10;

7. 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно: идентификатор пользователя; число значков; место в рейтинге — чем больше значков, тем выше рейтинг. Пользователям, получившие одинаковое количество значков, присваивается одно и то же место в рейтинге. Сортировка по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH user_badge_counts AS (
    SELECT u.id AS user_id, COUNT(*) AS badge_count
    FROM stackoverflow.users u
    JOIN stackoverflow.badges b ON u.id = b.user_id
    WHERE date_trunc('day', b.creation_date)::date BETWEEN '2008-11-15' AND '2008-12-15'
    GROUP BY u.id
)
SELECT user_id, badge_count,
       DENSE_RANK() OVER (ORDER BY badge_count DESC) AS rank
FROM user_badge_counts
ORDER BY badge_count DESC, user_id
LIMIT 10;

8. Среднее количество очков на каждый пост пользователя. Таблица включает: заголовок поста; идентификатор пользователя; число очков поста; среднее число очков пользователя за пост, округлённое до целого числа. Посты без заголовка, а также те, что набрали ноль очков — не учитываются.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg_score
FROM (
    SELECT *
    FROM stackoverflow.posts
    WHERE title IS NOT NULL AND score != 0
) filtered_posts;

9. Заголовки постов, которые были написаны пользователями, получившими более 1000 значков.


In [ ]:
WITH UsersWith1000Badges AS (
    SELECT u.id AS user_id
    FROM stackoverflow.users u
    JOIN stackoverflow.badges b ON u.id = b.user_id
    GROUP BY u.id
    HAVING COUNT(b.id) > 1000
)
SELECT p.title
FROM stackoverflow.posts p
JOIN UsersWith1000Badges u ON p.user_id = u.user_id
WHERE p.title IS NOT NULL;

10. Данные о пользователях из Канады (англ. Canada) с разделением на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присваивается группа 1;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группа 2;
* пользователям с числом просмотров меньше 100 — группа 3.

In [ ]:
SELECT id AS user_id,
       views AS profile_views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views >= 100 THEN 2
           ELSE 3
       END AS user_group
FROM stackoverflow.users
WHERE location LIKE '%Canada%'
  AND views > 0;

11. Дополнение предыдущего запроса. Отображение лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе с сортировкой по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH t1 AS (
    SELECT id,
           views,
           CASE
               WHEN views >= 350 THEN 1
               WHEN views < 350 AND views >= 100 THEN 2
               ELSE 3
           END AS category
    FROM stackoverflow.users
    WHERE views != 0 AND location LIKE '%Canada%'
),
t2 AS (
    SELECT category,
           MAX(views) AS max_views
    FROM t1
    GROUP BY category
)
SELECT t1.id AS user_id,
       t1.views AS profile_views,
       t1.category AS user_group
FROM t1
JOIN t2 ON t1.category = t2.category AND t1.views = t2.max_views
ORDER BY t1.views DESC, t1.id;

12. Ежедневный прирост новых пользователей в ноябре 2008 года. Таблица включает: номер дня; число пользователей, зарегистрированных в этот день; сумму пользователей с накоплением.

In [ ]:
SELECT EXTRACT(DAY FROM creation_date) AS day_number,
       COUNT(*) AS daily_new_users,
       SUM(COUNT(*)) OVER (ORDER BY EXTRACT(DAY FROM creation_date)) AS accumulated_users
FROM stackoverflow.users
WHERE EXTRACT(MONTH FROM creation_date) = 11 AND EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY EXTRACT(DAY FROM creation_date)
ORDER BY day_number;

13. Для каждого пользователя, который написал хотя бы один пост, найден интервал между регистрацией и временем создания первого поста. Выведен идентификатор пользователя и разница во времени между регистрацией и первым постом.

In [ ]:
WITH FirstPost AS (
    SELECT p.user_id,
           p.creation_date AS post_date,
           u.creation_date AS created_date,
           ROW_NUMBER() OVER(PARTITION BY p.user_id ORDER BY p.creation_date) AS row_num
    FROM stackoverflow.posts AS p
    JOIN stackoverflow.users AS u ON p.user_id = u.id
)

SELECT user_id,
       post_date - created_date AS interval
FROM FirstPost
WHERE row_num = 1;

## Написание запросов: вторая часть

1. Общая сумма просмотров у постов, опубликованных в каждый месяц 2008 года с сортировкой по убыванию общего количества просмотров.

In [ ]:
SELECT
  DATE_TRUNC('month', creation_date)::date AS month,
  SUM(views_count) AS total_views
FROM
  stackoverflow.posts
WHERE
  EXTRACT(year FROM creation_date) = 2008
GROUP BY
  month
ORDER BY
  total_views DESC;

2. Самые активные пользователи, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Для каждого имени пользователя выведено количество уникальных значений user_id. Сортировка по полю с именами в лексикографическом порядке.

In [ ]:
WITH creation_date_post AS (
    SELECT user_id, creation_date
    FROM stackoverflow.posts AS p
    LEFT JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id
    WHERE type = 'Answer'
    )
    
SELECT u.display_name, COUNT(DISTINCT user_id) ct
FROM stackoverflow.users AS u
JOIN creation_date_post AS p ON p.user_id = u.id
WHERE DATE_TRUNC('day', p.creation_date) ::date <= DATE_TRUNC( 'day', u. creation_date)::date + INTERVAL '1 month'
GROUP BY 1
HAVING COUNT (*) > 100
ORDER BY 1;

3. Количество постов за 2008 год по месяцам. Отобраны посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Сортировка по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('month', p.creation_date)::date AS date,
       COUNT(p.id)
FROM stackoverflow.posts AS p
WHERE p.user_id IN (
    SELECT DISTINCT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
      AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
)
GROUP BY date
ORDER BY date DESC;

4. Выгрузка данных о постах: идентификатор пользователя, который написал пост; дата создания поста; количество просмотров у текущего поста; сумма просмотров постов автора с накоплением. Сортировка по возрастанию идентификаторов пользователей, данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT
    p.user_id,
    p.creation_date,
    p.views_count,
    SUM(p.views_count) OVER (PARTITION BY p.user_id ORDER BY p.creation_date) AS cumulative_views
FROM stackoverflow.posts p
ORDER BY p.user_id, p.creation_date;

5. Среднее количество дней взаимодействия пользователей с платформой в период с 1 по 7 декабря 2008 года включительно (по публикации хотя бы одного поста).

In [ ]:
WITH user_activity AS (
    SELECT
        user_id,
        COUNT(DISTINCT DATE_TRUNC('day', creation_date)) AS active_days
    FROM stackoverflow.posts
    WHERE creation_date >= '2008-12-01' AND creation_date <= '2008-12-07'
    GROUP BY user_id
)
SELECT
    ROUND(AVG(active_days)) AS average_active_days
FROM user_activity;

6. Изменение количества постов ежемесячно с 1 сентября по 31 декабря 2008 года в процентах по сравнению с предыдущим.

In [ ]:
WITH post_counts AS (
    SELECT
        DATE_TRUNC('month', creation_date) AS month,
        COUNT(*) AS post_count
    FROM stackoverflow.posts
    WHERE creation_date >= '2008-09-01' AND creation_date <= '2008-12-31'
    GROUP BY month
)
SELECT
    EXTRACT(MONTH FROM month) AS month_number,
    post_count,
    ROUND(((post_count - LAG(post_count) OVER (ORDER BY month))::numeric / LAG(post_count) OVER (ORDER BY month)) * 100, 2) AS percent_change
FROM post_counts;

7. Активность пользователя, который опубликовал больше всего постов за все время с момента регистрации, в октябре 2008 года: номер недели; дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH best_user AS (
    SELECT
        p.user_id AS user_id
    FROM
        stackoverflow.posts AS p
    GROUP BY
        p.user_id
    ORDER BY
        COUNT(p.id) DESC
    LIMIT
        1
), all_post_best_user AS (
    SELECT
        p.creation_date AS date_time,
        EXTRACT('week' FROM p.creation_date::DATE) AS week_numb
    FROM
        stackoverflow.posts AS p
    JOIN best_user AS bu ON bu.user_id = p.user_id
    WHERE
        EXTRACT('month' FROM p.creation_date::DATE) = 10
    ORDER BY
        week_numb
)
SELECT
    DISTINCT apbu.week_numb,
    MAX(apbu.date_time) OVER (PARTITION BY apbu.week_numb) AS last_post_date
FROM
    all_post_best_user AS apbu
ORDER BY
    apbu.week_numb;